In [1]:
import tensorflow.keras as keras
import numpy as np
import importlib

import kaggle_functions as kaggle
import model10_wideresnet
importlib.reload(kaggle); 
importlib.reload(model10_wideresnet);
from model10_wideresnet import Model

model_number = 'model10'

### Create datasets

In [2]:
complete_dataset, train_dataset, valid_dataset, valid_labels = kaggle.load_train_as_dataset(return_complete_set=True)

batch_size = 32
train_dataset_augmented, epoch_length = kaggle.augment_dataset(train_dataset, batch_size)

### Train new model

In [ ]:
Model().build(None).summary()

In [33]:
# Fit model
# See {model}.py for specific training instructions, like epochs or valid_patience
model_name = 'WideResNet_1'

model, history = kaggle.train_model(
    Model().build(None), train_dataset_augmented, valid_dataset, 
    epochs=100, valid_patience=20, epoch_length=epoch_length)

Epoch 1/100
298/298 [==============================] - 456s 2s/step - loss: 2.4451 - accuracy: 0.2119 - val_loss: 2.3330 - val_accuracy: 0.2944 - lr: 0.0010
Epoch 2/100
298/298 [==============================] - 395s 1s/step - loss: 2.2757 - accuracy: 0.2701 - val_loss: 4.2502 - val_accuracy: 0.1686 - lr: 0.0010
Epoch 3/100
298/298 [==============================] - 356s 1s/step - loss: 2.1978 - accuracy: 0.2965 - val_loss: 3.2866 - val_accuracy: 0.2624 - lr: 0.0010
Epoch 4/100
298/298 [==============================] - 356s 1s/step - loss: 2.1371 - accuracy: 0.3133 - val_loss: 4.1890 - val_accuracy: 0.1447 - lr: 0.0010
Epoch 5/100
298/298 [==============================] - 356s 1s/step - loss: 2.1032 - accuracy: 0.3165 - val_loss: 3.6578 - val_accuracy: 0.1829 - lr: 0.0010
Epoch 6/100
298/298 [==============================] - 356s 1s/step - loss: 2.0426 - accuracy: 0.3369 - val_loss: 2.5483 - val_accuracy: 0.2649 - lr: 0.0010
Epoch 7/100
298/298 [==============================] - 356

In [ ]:
# Save model
model.save(f'models/{model_number}/{model_name}')

# Plot model statistics during training
kaggle.plot_model_history(history, [['accuracy', 'val_accuracy'], ['loss', 'val_loss']])

### Fine-tune

In [ ]:
# Fine-tune model
fine_model, history = kaggle.fine_tune_model_filepath(
    f'models/{model_number}/DeeperVGG_6_84',
    complete_dataset.batch(32), valid_dataset, 
    epochs=3, learning_rate=0.00001)

In [11]:
# Save fine-tuned model
fine_model_name = 'DeeperVGG_7'
fine_model.save(f'models/{model_number}/{fine_model_name}')

INFO:tensorflow:Assets written to: models/model7/DeeperVGG_7\assets


### Evaluate

In [ ]:
model_name = ''
model = keras.models.load_model(f'models/{model_number}/{model_name}')
test_pred = np.argmax(model.predict(valid_dataset.batch(128)), axis=1)

kaggle.print_accuracy(valid_labels, test_pred)
kaggle.plot_confusion_matrix(valid_labels, test_pred)

### Generate test predictions

In [ ]:
model_name = ''
kaggle.generate_test_pred_filepath(f'models/{model_number}/{model_name}')

### Hypertune models

In [ ]:
tuner = kaggle.hypertune_model(
    Model(), train_dataset_augmented, valid_dataset, 
    model_number, 'hypertuner2021-11-30', epochs=200, trials=5)

In [ ]:
tuner = kaggle.load_hypertuner(Model(), model_number, '')